<img src="http://www.burntx.com/wp-content/uploads/2018/01/voting-09-768x384.jpg" align="center">

# <center><font color = #2E5266>Systematic and Statistical Uncertainty </font></center>
**<center>The 2016 US Presidential Election and Uncertainty: How did the pollsters get it wrong?</center>**
***
Since 2008, statistician Nate Silver's website, **<font color=#2E5266>fivethirtyeight.com</font>** has garnered praise for on a wide variety of topics in current politics and political news. Both the website and founder are most famous for correctly predicting election results -- most significantly the 2012 election in which fivethirtyeight (538) correctly predicted the US election outcome for all 50 sates and the District of Columbia. The same statistical model predicted Hillary Clinton as the winner of the 2016 US Presidential election, however we all know this to not be the reality. So what happened exactly? This notebook will explore the following questions:

- How did FiveThirtyEight make its predictions?
- Was the FiveThirtyEight prediction incorrect due to statistical error, systematic error, or both?
- Should we say that FiveThirtyEight and other major election analysts "failed"?

Before we begin, recall the definitions of **<font color = #6E8898>systematic</font>** and **<font color = #6E8898>statistical error</font>**:

>**<font color = #6E8898>Systematic error</font>** is is consistent, repeatable error associated with faulty equipment or a flawed experiment design while **<font color = #6E8898>statistical error</font>** is the unknown difference between a calculated value and the true value.

In the context of the 2016 general election, what would systematic vs statistical errors look like?

In [2]:
# Hex codes for color palette
# dark blue 2E5266
# lighter blue 6E8898
# blue gray 9FB1BC
# gray pink D3D0CB
# yellow accent E2C044

<font color=#6E8898>**Comment**</font>: Possibility of issue using widgets. Had to install certain dependencies and packages in order to be able to view the widgets on JupyterLab; might not be the case with datahub. Will have to check with Keeley!

**Question 1**&nbsp;&nbsp;&nbsp;&nbsp;Where could there be sources of statistical error?

In [3]:
# Replace this text box with your answer

**Question 2**&nbsp;&nbsp;&nbsp;&nbsp;What about systematic error?

In [4]:
# Replace this text box with your answer

### <center><font color = #2E5266>Overview of Prediction Failure</font></center>
<img src="https://localtvktvi.files.wordpress.com/2016/12/promo305512065.jpg?quality=85&strip=all" align="center"/>

The date is November 9th, 2016. The country is in a flurry as -- seemingly against all the data -- Hillary Clinton loses the 2016 general election. Newsweek [prematurely sends out copies](http://nypost.com/2016/11/09/national-recall-after-newsweek-misfires-with-clinton-cover/) of a magazine depicting Clinton as the country's new Madam President. A multitude of newssites print stories with headlines such as [How Data Failed Us in Calling an Election](https://www.nytimes.com/2016/11/10/technology/the-data-said-clinton-would-win-why-you-shouldnt-have-believed-it.html) and [Why 2016 Election Polls Missed Their Mark](https://www.pewresearch.org/fact-tank/2016/11/09/why-2016-election-polls-missed-their-mark/). In the latter article, Pew Research had the following to say:
> The results of Tuesday’s presidential election came as a surprise to nearly everyone who had been following the national and state election polling, which consistently projected Hillary Clinton as defeating Donald Trump. Relying largely on opinion polls, election forecasters put Clinton’s chance of winning at anywhere from 70% to as high as 99%

<center>When typing "2016 election predictions" into Google's search bar, the 4th and 6th autocomplete entries are "fail" and "wrong" respectively</center>
<img src="./assets/google.png" align="center"/>

A New York Times Upshot article <a href="https://www.nytimes.com/interactive/2016/upshot/presidential-polls-forecast.html">provided a roundup of election predictions</a> by news organization. They all predicted a Clinton win, but in varying degrees. Let's take a look!

<img src="./assets/hillary_polls.png" align="center"/>

**TODO** Possible discussion here?

Before we begin, we will be loading libraries to help us process and present tables of information. Don't be intimidated by the code in the cell below, you can simply run it by pressing `shift` + `enter`.

In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import ipywidgets as widgets

**TODO** What would a good segue here be?

As mentioned before, **fivethirtyeight** has long been esteemed for their uncannily accurate predictions. Their algorithms remain a "secret sauce" -- known only to Nate Silver, but what we can do is try and replicate as best we can the polling method employed and work onwards from there. Before we do so, however, we must consider the polling methodology. How can we be so sure that the polling method is sound?

A NYT article by the title *<a href="https://www.nytimes.com/2019/07/02/upshot/online-polls-analyzing-reliability.html">No One Picks Up the Phone, but Which Online Polls Are the Answer?</a>* describes this dilemma as follows:
>A decade ago, it was fairly easy to tell the difference between a good poll and a bad poll. Most pollsters assumed that high-quality survey research required a high-quality survey design: a so-called probability sample, meaning that everyone in the public had an equal chance of being included in a poll. A poll was either a good probability sample (from random dialing of telephone numbers, for example) or it was not (like an online poll).

Nowadays most pollsters rely on statistical adjustments known as "weighting" to correct their initial samples. Such is the case in the small case study below. We will be taking a look at a data set of voter demographics from the 2012 election and taking a deeper look into how and what kinds of statistical and systematic errors can affect interpretation.

In [134]:
presidential_polls = pd.read_csv('./data/2012_election_weekend.csv').drop(['psraid'],axis=1)
presidential_polls['educ']=pd.to_numeric(presidential_polls.educ)
presidential_polls = presidential_polls[presidential_polls['oftvote'] != ' ']
presidential_polls['oftvote']=pd.to_numeric(presidential_polls.oftvote)
presidential_polls.head()

,state,community,region,registered,oftvote,vote,sex,age,educ,racethn,marital,parent,income,party
0,23,U,1,1,2,1,2,52,4,1,3,1,3,2
1,12,S,3,1,1,1,2,68,7,1,2,2,7,3
2,51,U,3,1,1,1,2,43,5,2,1,2,6,2
3,45,U,3,1,1,2,1,39,4,1,1,2,9,1
4,18,S,2,1,1,1,2,78,8,1,1,2,6,2


After running the cell above, you should be looking at the first five entries of a <a href="https://www.people-press.org/dataset/november-2012-election-weekend-survey/"> 2012 Election Weekend poll</a> taken by the Pew Research Center. Below we have provided a key (or "dictionary") with what each column represents as well as all the possible values.

| Column | Description | Possible Values |
|--------|-----------------|-----------------|
| state  | State FIPS code | |
| community | Community type | R = rural<br>S = suburban<br>U = urban|
| region | US Census region | 1 = Northeast<br>2 = Midwest<br>3 = South<br>4 = West |
| registered | Registered to vote | 1 = Registered/Plan To/North Dakota<br>2 = Not Registered |
| oftvote | How often do you vote? | 1 = always<br>2 = Nearly Always<br>3 = Part of the Time<br>4 = Seldom<br>5 = Never |
| vote | Presidential choice | 1 = Obama/lean Obama<br>2 = Romney/lean Romney<br>3 = Other<br>9 = Refused/Don't Know |
| sex | Sex | 1 = male<br>2 = female |
| age | Age | # = Age<br>97 = 97 or above<br>99 = Refused/Don't Know |
| educ | Level of Education | 1 = Less than HS<br>2 = Some HS<br>3 = HS grad<br>4 = Some college<br>5 = Two-Year degree<br>6 = Four-Year degree<br>7 = Some post grad<br>8 = Post-grad degree<br>9 = Refused/Don't Know |
| racethn | Race/Ethnicity | 1 = White, non-Hisp<br>2 = Black, non-hisp<br>3 = Hispanic<br>4 = Other<br>9 = Refused/Don't Know |
| marital | Marital Status | 1 = Married<br>2 = Living w/ Partner<br>3 = Divorced<br>4 = Separated<br>5 = Widowed<br>6 = Never been married<br>9 = Refused/Don't Know |
| parent | Parent or guardian | 1 = Y<br>2 = N<br>9 = Refused |
| income | Annual Income | 1 = <10k<br>2 = 10-20k<br>3 = 20-30k<br>4 = 30-40k<br>5 = 40-50k<br>6 = 50-75k<br>7 = 75-100k<br>8 = 100-150k<br>9 = >150k<br>10 = Refused/Don't Know |
| party | Party Preference | 1 = Republican<br>2 = Democrat<br>3 = Independent<br>4 = No preference<br>5 = Other party<br>9 = Refused/Don't Know |

Let's take a look at who we expect to win the 2012 presidential electon given our dataset's 'oftvote' column:

In [19]:
# Just run this cell!
def poll_results(df):
    p = df['vote'].value_counts(normalize=True) * 100
    return p
poll_results(presidential_polls)

1    51.323722
2    39.685452
9     6.238532
3     2.752294
Name: vote, dtype: float64

Given our key above, we notice that the majority of voters seem to be leaning Democratic, with a vote preference for Obama. In reality, the 2012 election was indeed won by President Obama 51.1% to Romney's 47.2%! Coincidentally, the percentage seen here is very close to what is accepted as the 2012 election's popular vote.

***

**TODO**: Should I go over bootstrapping? Also, how familiar are students with the terms convenience sampling, etc.

Awesome! Now that we have our dataset ready for action, let's simulate taking samples! Our dataset contains only 3,815 entries. This may seem large but considering the population of the United States, it is a very very small sample of our total population. This is why taking a sample *representative* of the general population is so important. A sampling method is called **biased** if it systematically favors some outcomes over others. Bias can be intentional, but often it is not.

Let us consider, however, a situation in which a pollster collects data from everyone on a University campus. How might that skew what we expect from election results? Use the widget below to find out! Clicking a checkbox will alter the way a sample is taken from our "population".

In [167]:
# Functions that simulate different methodologies
# Include descriptions
def convenience_parents():
    p = presidential_polls[presidential_polls['parent'] == '1']
    return poll_results(p)
def convenience_rural():
    p = presidential_polls[presidential_polls['community'] == 'R']
    return poll_results(p)
def convenience_higheredu():
    p = presidential_polls[presidential_polls['educ'] > 6]
    return poll_results(p)
def convenience_ca():
    p = presidential_polls[presidential_polls['state'] == 6]
    return poll_results(p)
def convenience_al():
    p = presidential_polls[presidential_polls['state'] == 1]
    return poll_results(p)
def frequent_voters():
    p = presidential_polls[presidential_polls['oftvote'] < 3]
    return poll_results(p)
def registered_ind():
    p = presidential_polls[presidential_polls['party'] == 3]
    return poll_results(p)
def srs():
    p = presidential_polls.sample(n=200)
    return poll_results(p)

convenience_parents()

1    47.739362
2    44.680851
3     4.920213
9     2.659574
Name: vote, dtype: float64

In [168]:
# Creating widgets
# Convenience
# Change descriptions on buttons!
descriptions = ['condition1', 'condition2', 'condition3', 'condition4', 'condition5', 'condition6']
items = [widgets.Button(description=desc, layout=widgets.Layout(width='100%')) for desc in descriptions]
for i in range(0, len(descriptions)):
    items[i].style.button_color = '#9FB1BC'
left_box = widgets.VBox([items[0], items[1], items[2]])
right_box = widgets.VBox([items[3], items[4], items[5]])
convenience = widgets.HBox([left_box, right_box])

# Linking checkbox and action
def on_button_click(b):
    convenience_parents()
out = widgets.Output()
items[0].on_click(on_button_click)
convenience

**TODO**: Mess with widget formatting to allow for titles for the different categories. Anything to add? (Ask Sarah and Emlen)

Interesting! As you might have noticed, the sampling methods can alter the predicted election results by a little to a lot! 

### To be implemented here: Analysts' methodology
    
use FiveThirtyEight as example

1. import data
2. reproduce FiveThirtyEight's model 
3. show step by step how the model works

### To be implemented: dicussion